In [1]:
import cv2 # to use the webcam

In [2]:
%cd ../
%cd weights # notebook in a weird place so need to get to where the weights, .cfg and obj.name files are.
# Dont run if files are in the same directory as the notebook

/home/xilinx/jupyter_notebooks


In [3]:
# loads the weights and .cfg to run yolo
net = cv2.dnn.readNet('yolov3_5_gestures.weights','yolov3_5_gestures.cfg')

In [4]:
classes = [] # Gets our Object names
with open("obj_5_gestures.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [5]:
# Additional libraries needed
import numpy as np
import time
layer_names = net.getLayerNames()
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [8]:
# Since we are using the PYNQ board we need the bitstream to access the parts of the board
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
base = BaseOverlay("base.bit")

In [9]:
# monitor configuration: 640*480 @ 60Hz
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

# monitor (output) frame buffer size
frame_out_w = 1920
frame_out_h = 1080
# camera (input) configuration
frame_in_w = 640
frame_in_h = 480

In [11]:
import os
os.environ["OPENCV_LOG_LEVEL"]="SILENT"
# initialize camera from OpenCV

videoIn = cv2.VideoCapture(0)
#videoIn = cv2.cvtColor(cv2.COLOR_BGR2RGB)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);


print("Capture device is open: " + str(videoIn.isOpened()))

[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (616) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Capture device is open: True


In [10]:
# No need to run this unless you are trying to test if your webcam + hdmi output are working properly

# Capture webcam image
import numpy as np
while True:
    ret, frame_vga = videoIn.read()

    # Display webcam image via HDMI Out
    # Display webcam image via HDMI Out
    if (ret):      
        outframe = hdmi_out.newframe()
        outframe[0:480,0:640,:] = frame_vga[0:480,0:640,:]
        hdmi_out.writeframe(outframe)
    else:
        raise RuntimeError("Failed to read from camera.")

KeyboardInterrupt: 

In [12]:
# Here is the main code for running yolov3 on the PYNQ board

font = cv2.FONT_HERSHEY_PLAIN
frame_id = 1


while True:
    starting_time = time.time()
    ret, frame = videoIn.read()
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)
    
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.85:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), font, 1, color, 2)  

    elapsed_time = time.time() - starting_time
    fps = frame_id / elapsed_time
    cv2.putText(frame, "FPS: " + str(round(fps, 2)), (10, 50), font, 4, (0, 0, 0), 3)
    if (ret):      
        outframe = hdmi_out.newframe()
        outframe[0:480,0:640,:] = frame[0:480,0:640,:]
        hdmi_out.writeframe(outframe)
    key = cv2.waitKey(1)
    #frame_id = frame_id + 1
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 